<a href="https://colab.research.google.com/github/MarcoMinchella/SurfNLP/blob/main/Milliaud_Minchella_NLP_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
  <h1 align="center">Retrieval-Augmented Generation (RAG) applied to surf spot chatbot</h1>
  <p align="center">
    <a href="https://github.com/AirMark14">Marco Minchella (2131787)</a>
    ·
    <a href="https://CharlesMld.github.io/">Charles Milliaud (2129400)</a>
  </p>
  <h3 align="center">| <a href="https://github.com/MarcoMinchella/SurfNLP/tree/main/data">Dataset</a> |</h3>
  <div align="center"></div>
</p>


In [ ]:
from IPython.display import Image
Image(url="https://www.surfertoday.com/images/stories/surfer-robot.jpg")

In [ ]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"
!pip install -U langchain-community
!pip install -q -U faiss-cpu tiktoken sentence-transformers # FAISS embedding model
!pip install huggingface-hub -q # Install hugging face from which we'll import llama LLM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

# Data preparation
**File format :**\
user,title,text,rating,url

For our project we decided to take as domain the reviews for the best 20 surfing spots in the world.\
Let's talk a little bit about the dataset construction.
The dataset was built using reviews from TripAdvisor, Google Maps and Yelp which were then put in an Excel file via a scraper that allowed us to avoid the work of copying and pasting all data by hand.
Every beach has around 150 reviews, most of them are from TripAdvisor, but whenever we couldn't get enough data we used Google Maps and in rare instances Yelp. We decided to include for each review the username of the author, the title of the review, the text of the review, the url and the rating (if present). The text of the reviews spanned from a single word to multiple paragraphs and contain alfanumeric data,however some of the reviews from Google Maps had to be discarded since they provided us with only blank space. In total we collected 2765 reviews.

After some tests we noticed that the embedding model and LLM didn't take into account some important metadata (surfspot name). So for example, if we asked the model about a specific spot it would only take into account the reviews that mentioned this spot in their actual review text. However, some reviews as the one below :\
\
user: Joana Duarte\
title: Praia dos Supertubos\
text: Clean, spacious, wonderful sand. A little windy\
rating: \
url:...\
\
Don't mention the spot name in the review text. Therefore, it would be impossible for the model to actually know which spot the review is about. Therefore, we wrote a small script to add at the beginning of each review the spot name about which the review is about. For example :\
\
user: Joana Duarte\
title: Praia dos Supertubos\
text: Supertubos Peniche Clean, spacious, wonderful sand. A little windy\
rating: \
url:...\
\
We will later compare the results of these 2 approaches

In [ ]:
!wget https://raw.githubusercontent.com/AirMark14/BestSurfingBeaches/main/NewCSVTagged_NLP/AllReviews.csv
from langchain.document_loaders.csv_loader import CSVLoader
all_loader = CSVLoader(file_path="AllReviews.csv", source_column="url") # reading csv file with the class "url" as key
all_data = all_loader.load() # convenient list of all the reviews (banzai_data[O] being the first one etc...)
print("Number of reviews :", len(all_data))
# Displaying some reviews
for i in range(2690,2700):
  print(all_data[i].page_content)

--2024-06-21 19:03:23--  https://raw.githubusercontent.com/AirMark14/BestSurfingBeaches/main/NewCSVTagged_NLP/AllReviews.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1126097 (1.1M) [text/plain]
Saving to: ‘AllReviews.csv’

AllReviews.csv      100%[===================>]   1.07M  --.-KB/s    in 0.03s   

2024-06-21 19:03:23 (31.6 MB/s) - ‘AllReviews.csv’ saved [1126097/1126097]

Number of reviews : 2764
user: Joana Duarte
title: Praia dos Supertubos
text: Supertubos Peniche Clean, spacious, wonderful sand. A little windy
rating: 
url: https://www.google.com/maps/reviews/data=!4m8!14m7!1m6!2m5!1sChdDSUhNMG9nS0VJQ0FnSUQ4ZzdXdjNBRRAB!2m1!1s0x0:0x3b1aea93b7d5a36d!3m1!1s2@1:CIHM0ogKEICAgID8g7Wv3AE%7CCgwImsz7-AUQ2LnB5AE%7C?hl=en
user: Bruno Violante
title: Pr

Here we basically split the list of documents into smaller chunks of 1000 characters each with an overlap of 100 characters (text length is counted using the len function). And we are storing the chunked documents in all_documents. We obtain 2873 chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,     # the character length of the chunk
    chunk_overlap = 100,   # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

all_documents = text_splitter.transform_documents(all_data)
print("Number of chunks :", len(all_documents))
print("Content of the 2752-th chunk :", all_documents[2752]) # printing the 2000's review

Number of chunks : 2882
Content of the 2752-th chunk : page_content='user: Rui Horta\ntitle: Praia dos Supertubos\ntext: Supertubos Peniche Nice beach a bit windy.\nrating: \nurl: https://www.google.com/maps/reviews/data=!4m8!14m7!1m6!2m5!1sChdDSUhNMG9nS0VJQ0FnSUNxeVBUdGpBRRAB!2m1!1s0x0:0x3b1aea93b7d5a36d!3m1!1s2@1:CIHM0ogKEICAgICqyPTtjAE%7CCgwI6L6yhQYQkMCuiwE%7C?hl=en' metadata={'source': 'https://www.google.com/maps/reviews/data=!4m8!14m7!1m6!2m5!1sChdDSUhNMG9nS0VJQ0FnSUNxeVBUdGpBRRAB!2m1!1s0x0:0x3b1aea93b7d5a36d!3m1!1s2@1:CIHM0ogKEICAgICqyPTtjAE%7CCgwI6L6yhQYQkMCuiwE%7C?hl=en', 'row': 2646}


After using different chunk sizes, we see that too little chunk size makes the review content disappear sometimes since title, url and author already takes a lot of space. We therefore decided to stick to the original parameters.

With our documents transformed into more manageable sizes, and with the correct
metadata set-up, we move on to creating our VectorStore.

# Creating an index
We'll use Facebook AI Similarity Search (FAISS) VectorStore, a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. The vector store is important but not as much as the sentence embedding model we will use. In fact, the VectorSore is a way to store the embeddings (vectors) created by the sentence embedding model.

We're going to be setting up our VectorStore with an embedding model from the famous sentence-transformers library. While this embedding model does not need to be the same as the LLM that we will use later, we still need to embed the index with the same model as the one we will use to embed the queries over that index.

>We're going to leverage a CacheBackedEmbeddingsflow to prevent us from re-embedding similar queries over and over again. Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application. **Particularly useful when users ask multiple times similar queries**

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")
embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embedding_model
)

embedder_parameters = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embedding_model
)

vector_store = FAISS.from_documents(all_documents, embedder_parameters)
print(type(vector_store))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<class 'langchain_community.vectorstores.faiss.FAISS'>


We basically created our vectorstore here. Let's see how it works by embedding a query and retrieving passages from our reviews that are close to it. Below we can see that it's a 2882x384 vector and it should contain our database.

In fact, we clearly see that the number of line in the same as the number of chunks. Therefore, each chunks is characterized by 384 values (parameters).

Something that we could do is to manually compare the euclidean distance between the vector of the query each one of the index and compute ourselves the closest one (most similar).

In [ ]:
print(vector_store.index.reconstruct_n().shape)
query = "What is chocolate ?"
embedding_vector = core_embeddings_model.embed_query(query)
# Taking a look at the query embeddings
print(embedding_vector)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 5)
print(docs[1].page_content)

(2882, 384)
[-0.015378851443529129, -0.024877436459064484, 0.004930006340146065, 0.1309206485748291, -0.07298801094293594, 0.058643314987421036, 0.05438821762800217, 0.02067364566028118, -0.0241529643535614, 0.03330467641353607, -0.033174820244312286, -0.06664614379405975, -0.05155558884143829, -0.02538890205323696, 0.00954887643456459, 0.014813819900155067, 0.043882448226213455, -0.010839988477528095, -0.005274939350783825, 0.013074384070932865, 0.03777680918574333, -0.04303635656833649, -0.04632044583559036, 0.04908052831888199, 0.03442646563053131, 0.07307764142751694, 0.022104520350694656, -0.08323477953672409, -0.0073641883209347725, 0.0054328348487615585, 0.030474666506052017, -0.06610441207885742, 0.11111718416213989, 0.03578365594148636, -0.09714415669441223, 0.04094745218753815, 0.003519096877425909, 0.0034977139439433813, -0.027187634259462357, -0.033491235226392746, -0.01831626333296299, 0.03579086437821388, 0.011799798347055912, -0.045159727334976196, 0.06776729971170425, 0

In [ ]:
# Compare 2 same queries to see how much time CacheBackedEmbeddings saves us
%%timeit -n 1 -r 1
query = "I enjoyed surfing on this beach!"
embedding_vector = embedder_parameters.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

12.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit
query = "I enjoyed surfing on this beach!"
embedding_vector = embedder_parameters.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

6.33 ms ± 138 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


These results show us how cached embeddings saves us a significant amount of time.

# Our LLM


Now that the retriever is done, we need to set up our LLM. We'll just use it to get docs that are relevant to our query and our query

Specifically, we'll be using: `meta-llama/Llama-2-13b-chat-hf`
It's a 13B parameter model that we're going to run on *less than* 15GB of GPU RAM,it's a fine-tuned model, optimized for dialogue use cases and converted for the Hugging Face Transformers format. We made the choice of using this LLM because it's one of the best open source LLM and according to the developers Llama-2-Chat models outperform open-source chat models on most benchmarks that they tested (source: [Huggingface site](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf)).





In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True, # load model in 4bit
    bnb_4bit_quant_type='nf4', #use nf4 (quantization storage format) type for that 4bit quantization
    bnb_4bit_use_double_quant=True, #we are going to quantize our quantizations constants
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

# Now we pipe it into a pipeline for compatibility with LangChain
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.5,   # temperature is creativity (risk is hallucination)
    max_new_tokens=256 # we don't want too much difference to the query
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

...

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)

# Getting the retriever out of our vector store
retriever = vector_store.as_retriever()

from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain.callbacks import FileCallbackHandler

handler = FileCallbackHandler("logs.txt")
#handler = StdOutCallbackHandler()

# Creates a question-answering chain that combines an llm (llm) and a document retriever (retriever)
qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler], # used to monitor or modify a chain behavior
    return_source_documents=False
)

In [ ]:
answer = qa_with_sources_chain({"query" : "Is surf good for health ?"})
print(answer['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The surf may be OK if thats all you came for otherwise its just a underdeveloped empty beach.
rating: 3
url: https://www.tripadvisor.com/ShowUserReviews-g616021-d1859639-r185263534-Riyue_Bay-Wanning_Hainan.html

user: andimarj
title: Good beach
text: Sasquerma Beach Clean, quiet, beautiful beach, but with strong sea, mts waves. Difficult for common bathing in certain regions, but said to be ideal for surfing.
rating: 3
url: https://www.tripadvisor.com/ShowUserReviews-g1096016-d4010884-r532989422-Praia_de_Itauna-Saquarema_State_of_Rio_de_Janeiro.html

user: walbs07
title: Great beach!
text: Sasquerma Beach Itaúna beach is not the most suitable for swimming, since there are many currents and strong waves, making it deep very quickly. So much so that it is widely used by surfers... However, it is clean, with clean sand, not too

In [ ]:
# Final chatbot answers extracted from
answer = qa_with_sources_chain({"query" : "Does surfing makes you mysogyne ?"})
def extract_helpful_answer(model_output):
  start_index = model_output.find("Helpful Answer:")
  if start_index == -1:
    return "No helpful answer found."
  else:
    end_index = model_output.find("\n", start_index)
    return model_output[start_index + len("Helpful Answer:"):end_index].strip()
extract_helpful_answer(answer['result'])

'Based on the reviews, it does not appear that surfing makes someone misogynistic. The reviews are written by people of different genders and backgrounds, and they all seem to have positive experiences at the beaches they visited. Some of the reviewers even mention their spouses or children accompanying them on their surfing trips. Therefore, it is not possible to conclude that surfing makes someone misogynistic based on these reviews'

# Llama-2-13b-chat-hf, temperature 0.1, text-generation

### What's a good surf spot in winter ? (without spot extensions in review text)
Based on the reviews, Snapper Rocks in Coolangatta, Australia is a great surf spot in winter. The waves are consistent and the water is clear and beautiful. However, it can get crowded during holidays, so it's best to avoid those times. Additionally, Banzai Pipeline in Pupukea, Oahu, Hawaii is another great spot for winter surfing, but it's important to be careful due to the powerful waves and strong currents.

*2nd try*\
Snapper Rocks in Coolangatta, Gold Coast, Queensland, Australia is a great surf spot in winter. It has consistent waves, clear water, and a large beach. However, it can get crowded during holidays.

### What's a good surfspot in winter ? (with spot extensions)
Based on the reviews, Banzai Pipeline Pipeline in Hawaii is a great surf spot in winter. TimDore recommends it as a must-see surf spot during the winter months, and the surf is incredibly powerful. However, it's important to be a strong swimmer and listen to the lifeguards.

### Where should I go to surf ? (without extensions)
If you like to surf, you should go to Wanning.

### Where should I go to surf ? (with extensions)
Based on the reviews, it seems like you're looking for a spot with big waves. TimDore's review of Banzai Pipeline Pipeline in Oahu, Hawaii suggests that it's a must-see surf spot during the winter months, but it can be flat during the summer. However, the waves are incredibly powerful, so it's important to be a strong swimmer and listen to the lifeguards.

Alternatively, if you're looking for a spot with big wave surfing, ulle's review of Peahi in Maui, Hawaii suggests that it's a great spot to watch big wave surfing. However, the road to the beach can be difficult to navigate, so it's important to have the right car or plan for a hitchhike.

Based on these reviews, it seems like you should consider visiting Oahu or Maui, Hawaii for big wave surfing. However, it's important to keep in mind the power of the waves and the difficulty of accessing the beaches, so be sure to plan accordingly and prioritize safety.

### What is a good spot for beginners ? (without spot extensions)
Based on the reviews, it seems that Sandy Beach in Rincon, Puerto Rico is a good spot for beginner surfers. According to the reviews, the beach has a sandy bottom, which is easier to surf on than rocky or coral reefs. Additionally, surf lessons are offered at the beach and the instructors are reportedly very nice

### What is a good spot for beginners ? (with spot extensions)
Based on the reviews, it seems that Sandy Beach in Rincon, Puerto Rico is a good spot for beginners. The review by lbisurfergirlnj mentions that the beach is good for beginner surfers and surf lessons are offered with nice instructors. Additionally, the beach has a sandy bottom, which is easier to surf on compared to a rocky or coral reef bottom.

### What's a good spot for intermediates ? (without spot extensions)
Based on the reviews, Snapper Rocks would be a good spot for intermediates. The wave is not too fast and has an open face for vertical surfing, making it suitable for those with some experience under their belt. Additionally, the local surf shop is great and offers decent selection of boards for rent, as well as hotel rooms at better rates

### What's a good spot for intermediates ? (with spot extensions)
Based on the reviews, Snappers Rock seems to be a good spot for intermediates. According to user themets2016, the wave is rippable and not overly fast, making it suitable for more experienced surfers to practice. Additionally, the local surf shop is great and has decent selection of boards for rent. However, it's important to note that the spot may be crowded and there are limited food options nearby.

### What's a good spot for confirmed level surfers ? (without spot extensions)
Based on the reviews, a good spot for confirmed level surfers would be Snapper Rocks in Coolangatta, Gold Coast, Queensland. According to Compass2101, it is one of the best surf spots in the world and offers safe options for kids. The right-hander sand bottom barrels when overhead, providing a great opportunity for experienced surfers to ride 2km down to Kirra on 3-4 waves

### What's a good spot for confirmed level surfers ? (with spot extensions)
Based on the reviews, Supertubos Peniche is a good spot for confirmed level surfers. According to yannilosg, the waves are powerful and experienced surfers can showcase their skills. However, it's important to respect the local surfers and be mindful of the conditions.

### What's chocolate ?
Chocolate is a sweet, brown, or white, edible substance that is made from cacao beans. It is often used as a flavoring or decoration for foods and drinks


###We are now going to give a query which is extremely odd. The reason we chose to do this is because while skimming through the dataset we found a very strange review of a particular user which had nothing to do with the place he went to visit but instead, with some strong words, just stated the dislike he has for his spouse. We wanted to see if we would get an answer based on that particular outlier review, if our system would give us a more nuanced answer or if it would just hallucinate:
### Does surfing makes you a misogynist ?
Based on the reviews, it does not appear that surfing makes someone misogynistic. The reviews are written by people of different genders and backgrounds, and they all seem to have positive experiences at the beaches they visited. Some of the reviewers even mention their spouses or children accompanying them on their surfing trips. Therefore, it is not possible to conclude that surfing makes someone misogynistic based on these reviews

 ---

 Let's talk a bit about the results that we obtain with these parameters (that should be the best for the desired output).

 First of, we can see the obvious difference between the model that's based on the raw review text and the one that has the review extended. This difference is clear when we look at the "Where should I go surf ?". Although some answers resemble others with or without extensions, the arguments are often better in the ones that have the extensions, furthermore by using a dataset with location extentions assures us to give some geographical context to all those reviews that do not mention the name of the spot.

 It's also interesting to take a look at some out of context queries. If we asked the embedding model what's chocolate it would answer with a review which has no link whatsoever but it's normal since it can only answer with the index. However, we notice that the LLM is way more accurate since :the pre-training is already done. We could even compare the answers of the LLM with/without the vector store. The answer on misogyny is also very interesting since we see how the LLM answers using the reviews and not his pretraining. The answer also shows how a single outlier in our dataset gets overlooked while the high number of reviews that show values and behavoirs of inclusivity prevale. Notice that even if none of the aforementioned reviews explicitly state words of inclusivity and gender equality, our system is still able to provide a negative answer to the question.

 Let's try on a 0.5 temperature for the same model.

 # Llama-2-13b-chat-hf, temperature 0.5, text-generation
On basic queries, the model doesn't seem to be very creative since the answers are pretty much the same as the previous one. However, when we ask some out-of-context questions like : Does surfing makes you a misogynist ? we get that answer: No, surfing does not make you misogynistic. The stereotype that surfers are misogynistic is a common one, but it is not supported by evidence. In fact, many surfers are known for their progressive and inclusive attitudes towards women and gender equality.

Which is not really using the dataset as we can see. However, overall the model seems surprisingly serious for a temperature of 0.5 and doesn't really propose out of the box answers.

 ---

To conclude on our tests, we can say that the most important part of a RAG system is the dataset since the already existing embedding and LLM models are really performant. However, we can see that the question answering time is consequent which isn't convenient for a chatbot. Some improvements are still to be made to our RAG system but the performances seem very accurate.